# Bronze層テーブル作成（CSV to Delta）

このノートブックでは、メダリオンアーキテクチャのBronze層を構築します。生のCSVファイルを読み込み、Deltaテーブル形式で保存します。

## 目的
- **データレイク基盤の構築**: 生データをそのまま保存
- **データの永続化**: CSVファイルをDeltaテーブルに変換
- **スキーマ推論**: データ型の自動判定と適用

## 処理対象
1. **transactions.csv** - 取引履歴データ
2. **customers.csv** - 顧客マスタデータ
3. **products.csv** - 商品マスタデータ
4. **stores.csv** - 店舗マスタデータ

## Bronze層の特徴
- 生データをそのまま保存（最小限の変換のみ）
- データの歴史と系譜を保持
- 後続のSilver/Gold層の元データとして機能


## 環境設定とカタログ/スキーマの切り替え

処理前に使用するカタログとスキーマを定義し、明示的に切り替えます。


In [0]:
# 環境変数の定義（環境に応じて変更）
catalog_name = "users"                      # Unity Catalogのカタログ名
schema_name = "yukiteru_koide"              # スキーマ名
base_path = "/Volumes/users/yukiteru_koide/yukiterumart_etl"  # CSVファイルが格納されたVolumeパス

# CATALOGとSCHEMAを明示的に切り替え
spark.sql(f"USE CATALOG {catalog_name}")    # カタログを指定
spark.sql(f"USE SCHEMA {schema_name}")      # スキーマを指定

print(f"カタログ: {catalog_name}, スキーマ: {schema_name} に切り替えました")


## CSVファイルの読み込みとBronze層Deltaテーブル作成

各CSVファイルを読み込み、スキーマを自動推論してDeltaテーブルとして保存します。

### 処理の流れ
1. CSVファイルを読み込み（ヘッダー付き、スキーマ自動推論）
2. Deltaフォーマットで保存（上書きモード）
3. Unity Catalogのテーブルとして登録


In [0]:
# 1. トランザクションデータの処理
print("📊 トランザクションデータを処理中...")
df_transactions = (
    spark.read.format("csv")                    # CSV形式で読み込み
    .option("header", True)                     # ヘッダー行あり
    .option("inferSchema", True)                # スキーマ自動推論
    .load(f"{base_path}/transactions.csv")     # ファイルパス指定
)
# Deltaテーブルとして保存（既存テーブルは上書き）
df_transactions.write.format("delta").mode("overwrite").saveAsTable("transactions")
print(f"✅ transactionsテーブル作成完了 - レコード数: {df_transactions.count()}")

# 2. 顧客データの処理
print("👥 顧客データを処理中...")
df_customers = (
    spark.read.format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load(f"{base_path}/customers.csv")
)
df_customers.write.format("delta").mode("overwrite").saveAsTable("customers")
print(f"✅ customersテーブル作成完了 - レコード数: {df_customers.count()}")

# 3. 商品データの処理
print("🛍️ 商品データを処理中...")
df_products = (
    spark.read.format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load(f"{base_path}/products.csv")
)
df_products.write.format("delta").mode("overwrite").saveAsTable("products")
print(f"✅ productsテーブル作成完了 - レコード数: {df_products.count()}")

# 4. 店舗データの処理
print("🏪 店舗データを処理中...")
df_stores = (
    spark.read.format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load(f"{base_path}/stores.csv")
)
df_stores.write.format("delta").mode("overwrite").saveAsTable("stores")
print(f"✅ storesテーブル作成完了 - レコード数: {df_stores.count()}")

print("\n🎉 Bronze層テーブル作成が完了しました！")

# レビュー
df_reviews = (
    spark.read.format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load(f"{base_path}/reviews.csv")
)
df_reviews.write.format("delta").mode("overwrite").saveAsTable("reviews")
